In [423]:
import numpy as np
import pandas as pd
import json
import datetime as dt
data = json.load(open("../backtest_results/backtest-result-2023-01-13_16-41-59.json"))
df = pd.DataFrame(data["strategy"])
coin = "XRP"

In [424]:
trades = df["VB_1d"]["trades"]
trades = pd.DataFrame(trades)
stake_amount = trades["stake_amount"]
print(stake_amount.median())
print(trades)

304.16353200000003
          pair  stake_amount  max_stake_amount    amount  \
0     BTC/USDT     191.33360         191.33360     0.020   
1     BTC/USDT     194.01282         194.01282     0.023   
2     BTC/USDT     186.72302         186.72302     0.022   
3     ETH/USDT     194.83497         194.83497     1.221   
4     BTC/USDT     191.97318         191.97318     0.021   
..         ...           ...               ...       ...   
195  DOGE/USDT     738.94485         738.94485  6405.000   
196   ETH/USDT     746.16260         746.16260     0.470   
197   SOL/USDT     741.00000         741.00000    20.000   
198  DOGE/USDT     761.57760         761.57760  5880.000   
199   ETH/USDT     727.00608         727.00608     0.552   

                     open_date                 close_date   open_rate  \
0    2019-10-26 04:00:00+00:00  2019-10-31 04:00:00+00:00  9566.68000   
1    2020-01-08 01:00:00+00:00  2020-01-13 01:00:00+00:00  8435.34000   
2    2020-01-14 06:00:00+00:00  2020-01-1

In [425]:
losses = trades.loc[trades["open_rate"] > trades["close_rate"]]
# print(losses)
new = pd.Series((losses["open_rate"] - losses["close_rate"])*losses["amount"])
# losses.loc["actual_loss"] = (losses["open_rate"] - losses["close_rate"])*losses["amount"]
# print(losses.columns)
print(new.mean())

36.83216059433962


In [426]:
winners = trades.loc[trades["open_rate"] < trades["close_rate"]]
new_1 = pd.Series(( winners["close_rate"] - winners["open_rate"] )*winners["amount"])

print(new_1.mean())

70.44609007446807


In [427]:

winners_more_than10 = trades.loc[ trades["close_rate"] > 1.05*trades["open_rate"] ]
winners_more_than10_doge = winners_more_than10.loc[winners_more_than10['pair'] == coin+"/USDT"]
winners_more_than10_doge_open_date = winners_more_than10_doge["open_date"]
print(winners_more_than10_doge_open_date)



37     2020-11-17 03:00:00+00:00
39     2020-11-22 03:05:00+00:00
46     2020-11-26 21:00:00+00:00
99     2021-01-30 20:00:00+00:00
120    2021-04-05 15:00:00+00:00
122    2021-04-07 09:15:00+00:00
124    2021-04-12 09:20:00+00:00
157    2021-08-11 06:00:00+00:00
179    2022-02-07 18:00:00+00:00
190    2022-09-18 21:00:00+00:00
Name: open_date, dtype: object


In [428]:
def round(x):
    etc = x%86400
    if(etc ==0):
        return x
    else: 
        return x-etc

In [429]:
winners_more_than10_doge_open_date = winners_more_than10_doge_open_date.apply(lambda x : dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z"))

winners_more_than10_doge_open_date = winners_more_than10_doge_open_date.apply(dt.datetime.timestamp)
winners_more_than10_doge_open_date = winners_more_than10_doge_open_date.apply(np.int64)
print(winners_more_than10_doge_open_date)
winners_more_than10_doge_open_date = winners_more_than10_doge_open_date.apply(lambda x: round(x) )
winners_more_than10_doge_open_date = winners_more_than10_doge_open_date.apply(lambda x: 1000*x)
winners_more_than10_doge_open_date.head()

37     1605582000
39     1606014300
46     1606424400
99     1612036800
120    1617634800
122    1617786900
124    1618219200
157    1628661600
179    1644256800
190    1663534800
Name: open_date, dtype: int64


37     1605571200000
39     1606003200000
46     1606348800000
99     1611964800000
120    1617580800000
Name: open_date, dtype: int64

In [430]:
hour_data = json.load(open(f"../data/binance/futures/{coin}_USDT-1d-futures.json"))

hour_df = pd.DataFrame(hour_data)

hour_df.head()

,0,1,2,3,4,5
0,1578268800000,0.1970,0.2259,0.1970,0.2226,92022218.9
1,1578355200000,0.2227,0.2297,0.2071,0.2138,124985943.9
2,1578441600000,0.2138,0.2204,0.2013,0.2078,79791597.2
3,1578528000000,0.2078,0.2089,0.2010,0.2040,45126429.9
4,1578614400000,0.2039,0.2136,0.2001,0.2128,62488677.4


In [431]:
hour_df['noise'] = 1- abs((hour_df.loc[:,1] - hour_df.loc[:,4])/
                                    (hour_df.loc[:,2] - hour_df.loc[:,3]))
        
        
   
# day_dataframe['noise_20_day_average'] = day_dataframe['noise'].rolling(20).mean()
hour_df['noise_5_day_average'] = hour_df['noise'].rolling(10).mean()
hour_df['noise_5_day_average'].median()

0.5911274709180794

In [432]:
# print(hour_df['noise_5_day_average'].loc[hour_df.iloc[:,0].isin(winners_more_than10_doge_open_date)])
print(hour_df['noise_5_day_average'].loc[hour_df.iloc[:,0].isin(winners_more_than10_doge_open_date)].median())

0.47741472780253313


In [433]:
# 5일
# XRP-> median 0.5849, winners 0.4241
# BTC-> 0.5803, 0.5334
# DOGE-> 0.6007, 0.4603
# SOL -> 0.5622, 0.5434
# ETH -> 0.5548, 0.5322


# 3일
# XRP 0.5906, 0.4183
# BTC 0.5823, 0.5304
# DOGE 0.6033, 0.4340
# SOL 0.5617, 0.5570
# ETH 0.5505, 0.4857

SyntaxError: invalid decimal literal (4283047527.py, line 9)